# Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

# Initial Setup

To begin I've chosen to focus on getting all of the stats from one game.

Due to personal preference I've chosen to look at the preseason game between the Dallas Fuel and the Houston Outlaws. The page on winstons lab for the game can be found here:

[Preseason: Fuel vs. Outlaws](https://www.winstonslab.com/matches/match.php?id=2367)

The individual match id for the site is 2367. We'll try to keep everything as modular as possible during prototyping.

In [ ]:
BASE_MATCH_URL = "https://www.winstonslab.com/matches/match.php"

In [ ]:
match_id = "2367"

In [ ]:
full_url = BASE_MATCH_URL + "?id=" + match_id

## Get the base html using requests

In [ ]:
response = requests.get(full_url)
html = BeautifulSoup(response.text, "lxml")

# Overview stats

In [ ]:
div = html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [ ]:
detail.text

Ok so this is empty because It loads the data after clicked detailed stats. I guess we'll have to use Selenium and PhantomJS

In [ ]:
from selenium import webdriver
import time

In [ ]:
driver = webdriver.Chrome("/Applications/chromedriver")

JK using Chrome. I need to find a chrome headless driver.

In [ ]:
driver.get(full_url)
time.sleep(5)

In [ ]:
detail_button = driver.find_element_by_class_name("showDetailedStatsBut")

In [ ]:
detail_button.text

In [ ]:
detail_button.click()

In [ ]:
detail_stats = driver.find_element_by_id("detailedStatsDiv_0")

In [ ]:
detail_stats.text

In [ ]:
page_source = driver.page_source

In [ ]:
new_html = BeautifulSoup(page_source, "lxml")

In [ ]:
div = new_html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [ ]:
stat_table = detail.table

In [ ]:
table_headers = stat_table.thead
table_data = stat_table.tbody

In [ ]:
header_elems = table_headers.findAll("th")

In [ ]:
header_list = []
for header in header_elems:
    header_list.append(header.text)
pprint(header_list)

In [ ]:
row_list = table_data.findAll("tr")

In [ ]:
full_stat_list = []

In [ ]:
for row in row_list[0:3]:
    elems = row.findAll("td")
    stat_list = []
    print("-----------------------")
    for i, item in enumerate(elems[0:]):
        print("{}: {}".format(header_list[i], item.text.strip()))
        stat_list.append(item.text.strip())
    full_stat_list.append(stat_list)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=header_list)

In [ ]:
df.head()

In [ ]:
full_stat_list

In [ ]:
new_df = pd.DataFrame(full_stat_list, columns=header_list)

In [ ]:
new_df.head()

In [1]:
import MatchStats as ms

In [2]:
df = ms.detailed_match_data("2367")

In [3]:
df.head()

,Player,Hero,Time,Rating,FWin%,PTK,K/10,D/10,U/10,TTCU,...,UE,UOOF,FK,FD,Impact,Elim/10,dmg/10,heal/10,ObjTime/10,ObjKills/10
0,Bani,Zenyatta,00:08:14,1451,55.56%,19.15%,10.93,6.07,4.86,91,...,44.44,25%,22%,11%,1355,N/A,N/A,N/A,N/A,N/A
1,Boink,Mercy,00:44:32,1206,55.32%,0.56%,0.22,3.82,3.59,87,...,21.6,12.5%,0%,9%,1116,N/A,N/A,N/A,N/A,N/A
2,Taimou,Roadhog,00:30:25,1193,57.14%,25.77%,8.22,4.6,4.27,74,...,5.36,8.33%,21%,11%,1098,N/A,N/A,N/A,N/A,N/A
3,Rawkus,Zenyatta,00:49:16,1181,62.75%,11.34%,4.47,5.07,4.87,92,...,6,5%,4%,4%,772,N/A,N/A,N/A,N/A,N/A
4,JAKE,Junkrat,00:28:21,1137,54.17%,38.37%,11.64,6.7,4.59,77,...,12.5,0%,33%,21%,1451,N/A,N/A,N/A,N/A,N/A


In [4]:
df.tail(12)

,Player,Hero,Time,Rating,FWin%,PTK,K/10,D/10,U/10,TTCU,...,UE,UOOF,FK,FD,Impact,Elim/10,dmg/10,heal/10,ObjTime/10,ObjKills/10
41,cocco,Orisa,00:02:37,NED,0%,16.67%,3.82,7.64,7.64,43,...,0,50%,0%,0%,1028,N/A,N/A,N/A,N/A,N/A
42,cocco,Reinhardt,00:03:34,NED,25%,7.69%,2.8,8.41,5.61,73,...,-25,50%,25%,25%,1335,N/A,N/A,N/A,N/A,N/A
43,clockwork,Mccree,00:04:21,NED,100%,33.33%,20.69,6.9,4.6,84,...,0,0%,67%,0%,1034,N/A,N/A,N/A,N/A,N/A
44,clockwork,Soldier: 76,00:00:14,NED,0%,0%,0,42.86,0,0,...,0,0%,0%,0%,991,N/A,N/A,N/A,N/A,N/A
45,LiNkzr,Tracer,00:04:21,NED,60%,52.94%,20.69,6.9,6.9,67,...,-10,0%,60%,0%,1281,N/A,N/A,N/A,N/A,N/A
46,Mickie,Winston,00:00:54,NED,0%,0%,0,22.22,0,0,...,0,0%,0%,0%,997,N/A,N/A,N/A,N/A,N/A
47,Boink,Zenyatta,00:01:06,NED,0%,16.67%,9.09,0,0,0,...,0,0%,0%,0%,998,N/A,N/A,N/A,N/A,N/A
48,Effect,Mccree,00:04:28,NED,0%,25%,6.72,17.91,4.48,80,...,0,0%,0%,50%,895,N/A,N/A,N/A,N/A,N/A
49,Effect,Reaper,00:02:12,NED,100%,30%,27.27,4.55,9.09,57,...,0,0%,50%,0%,996,N/A,N/A,N/A,N/A,N/A
50,Muma,Reinhardt,00:00:05,NED,0%,0%,0,0,0,0,...,0,0%,0%,0%,998,N/A,N/A,N/A,N/A,N/A
